<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Aug_6_Clinicalbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [115]:
import pandas as pd
import csv
path='/content/drive/My Drive/NLP/ClinicalBiobert/Clinicalbert-10000/BC2GM-IOB-Train_10000.xlsx'
data=pd.read_excel(path,nrows=10014,delimiter='\t',names=["Words","tags"],encoding='unicode_escape')

In [116]:
data.tail(50)

,Words,tags
9964,of,O
9965,acute,B-DISEASE
9966,Q,I-DISEASE
9967,fever,I-DISEASE
9968,(,O
9969,seroconversion,O
9970,and,O
9971,/,O
9972,or,O
9973,presence,O


In [117]:
data.shape

(10014, 2)

In [118]:
data['tags'].isnull().sum()

9

In [119]:
data['tags']=data['tags'].fillna('O')

In [120]:
data['tags'].isnull().sum()
data['tags'].unique()

array(['O', 'B-GENE', 'I-GENE', 'B-DISEASE', 'I-DISEASE', 'B-DRUGS',
       'I-DRUGS'], dtype=object)

In [121]:
data['tags'].value_counts()

O            8574
I-GENE        560
B-GENE        372
B-DRUGS       161
I-DRUGS       130
I-DISEASE     123
B-DISEASE      94
Name: tags, dtype: int64

Concatenate the words into a sentence - Make sure we convert by using str(input)

In [122]:
sentences1=[]
tags1=[]
words=data['Words'].values.tolist()
tag=data['tags'].values.tolist()
#words
temp_sen=[]
temp_tag=[]
count=0
for i in words:
  #print(i)
  
  if i != 'End':
    temp_sen.append(str(i))
    temp_tag.append(str(tag[count]))
    count+=1
  else:
    sentences1.append(temp_sen)
    tags1.append(temp_tag)
    temp_sen=[]
    temp_tag=[]

In [123]:
 #check the data type pf the sentences
 type(sentences1[0][2])

str

In [124]:
#Print the length of tags1 and sentences1
print(len(tags1))
print(len(sentences1))

321
321


In [125]:
#detect the maximum and mimim length of the sentences
l=[]
for i in sentences1:
  l.append(len(i))
print(max(l))
print(min(l))


139
2


In [126]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [127]:
#map the tags with integers -> tags refer to ner
#data["tag"]:retrieves unique tags, set(data["tag"]) -> refers to set of distinct values
tag_values=list(set(data["tags"].values))
tag_values.append("PAD")
#create a dictionary of tag and values
tag2idx={t:i for i,t in enumerate(tag_values)}

In [128]:
print(tag2idx)
#Tags available :
# I-GENE,B-GENE

{'O': 0, 'B-DRUGS': 1, 'I-GENE': 2, 'B-DISEASE': 3, 'I-DRUGS': 4, 'I-DISEASE': 5, 'B-GENE': 6, 'PAD': 7}


Set up the transformers library

In [129]:
!pip3 install transformers


In [130]:
#use pytorch implementation of bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(level=logging.INFO)

In [131]:
#import clinicalbiobert
scibert_tokenizer=AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#clibert_model=AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.tokenization_utils_base:Model name 'emilyalsentzer/Bio_ClinicalBERT' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased

In [132]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [171]:
#Set the max length and batchsize
#MAX_LEN = 64
Batch_Size = 32

In [23]:
#device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#n_gpu=torch.cuda.device_count()

In [134]:
def token_and_preserve_labels(sentence,text_labels):
  tokenized_sentence=[]
  labels=[]

  for word,label in zip(sentence,text_labels):
    #tokenize the word and count of # of subwords that word is toeknized into 
    #demonstrators ['demons', '##tra', '##tors']
    tokenized_word=scibert_tokenizer.tokenize(word)
    n_subwords=len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence,labels 

In [135]:
#sentence and corresponsing labels 
#sentence form : [list of words]
#labels form : #labels form [ list of ner tags]
tokenized_texts_and_labels=[token_and_preserve_labels(sent,labels) for sent,labels in zip(sentences1,tags1)]

In [136]:
print(tokenized_texts_and_labels)

[(['im', '##mu', '##no', '##his', '##to', '##chemical', 'stain', '##ing', 'was', 'positive', 'for', 's', '-', '100', 'in', 'all', '9', 'cases', 'stained', ',', 'positive', 'for', 'h', '##mb', '-', '45', 'in', '9', '(', '90', '%', ')', 'of', '10', ',', 'and', 'negative', 'for', 'c', '##yt', '##oker', '##ati', '##n', 'in', 'all', '9', 'cases', 'in', 'which', 'my', '##x', '##oid', 'me', '##lan', '##oma', 'remained', 'in', 'the', 'block', 'after', 'previous', 'sections', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DISEASE', 'B-DISEASE', 'B-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['ch', '##lora', '##mp', '##hen', '##ico', '##l', 'ace', '##ty', '##lt', '##ran', '##s', '##fer

In [137]:
# split text and labels
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [138]:
print(tokenized_texts[4])

['c', '##lon', '##ing', 'and', 'se', '##quencing', 'of', 'the', 'upstream', 'region', 'of', 'p', '##ep', '##x', 'revealed', 'the', 'presence', 'of', 'two', 'or', '##fs', 'of', '360', 'and', '1', ',', '33', '##8', 'b', '##p', 'that', 'were', 'shown', 'to', 'be', 'able', 'to', 'en', '##code', 'proteins', 'with', 'high', 'ho', '##mology', 'to', 'g', '##ln', '##r', 'and', 'g', '##ln', '##a', 'proteins', ',', 'respectively', '.']


In [139]:
print(labels[4])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'B-GENE']


In [140]:
input_ids = pad_sequences([scibert_tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],dtype="long", value=0.0,
                          truncating="post", padding="post")

In [141]:
print(len(input_ids))

321


In [38]:
#tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
#                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
#                     dtype="long", truncating="post")

In [142]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],value=tag2idx["PAD"], padding="post",dtype="long", truncating="post")

In [143]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [144]:
print(attention_masks)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1

In [145]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.3)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.3)

In [146]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [147]:
tr_tags.shape

torch.Size([224, 191])

In [148]:
val_tags.shape

torch.Size([97, 191])

In [149]:
tr_inputs.shape

torch.Size([224, 191])

In [172]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=Batch_Size)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=Batch_Size)

In [151]:
for input,mask,tag in train_dataloader:
  print(mask.shape)
  print(input.shape)
  print(tag.shape)

torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([64, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])


Setup the Bert model for finetuning

In [152]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import BertForTokenClassification,AdamW,AutoConfig
#from transformers import 

In [153]:
#Use small learning rates combined with bias correction to avoid vanishing gradients early in training.
#Increase the number of iterations considerably and train to (almost) zero training loss while
#making use of early stopping.

In [154]:
model1 = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embe

In [155]:
#model2=BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

In [156]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [157]:
model1.num_labels

8

In [158]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [159]:
#Print the different parameters
params=list(model1.named_parameters())

#print
print("the model has {:} number of parameters".format(len(params)))

the model has 201 number of parameters


In [160]:
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---First Transformer---")
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
for p in params[190:200]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---output layer---")
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)
---First Transformer---
bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.attention.output.LayerNorm.weight        (768,)
bert.encod

In [161]:
#grab the optimzer
optimizer=AdamW(model1.parameters(),#ξ=0.95 and lr=2.0e-5.
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

In [162]:
model1.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

In [163]:
from transformers import get_linear_schedule_with_warmup

In [164]:
epochs=2

In [165]:
total_steps = len(train_dataloader) * epochs

In [166]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)


In [167]:
from sklearn.metrics import f1_score, accuracy_score,classification_report

In [168]:
import random
import numpy as np

In [169]:
# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [173]:
#for ech epoch
#Run the training for the given epochs - typically between 2 and 4
#range 0 to 3

loss_values, validation_loss_values = [], []


for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  #Reset the total loss for this epoch
  total_train_loss=0
  
  #train the model
  model1.train()

  for step,batch in enumerate(train_dataloader):
    print(step)
    #print(batch)

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    #print(batch.shape)
    b_input_ids,b_input_mask,b_labels=batch
    #print(b_labels)
  #always clear the gradients before doing a backward pass
    model1.zero_grad()

   #perform a forward pass
    loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    
    #logits = logits.detach().cpu().numpy()
    #print("step number -", step)
    #print("score--",logits.shape)
    #outout shape in the form : 32*75*18 --32 batch size,75 --> sequence size, 18 --> Tags 
    #print("argmax--")
    #print("logits--",logits)
    #print("argmax--",np.argmax(logits,axis=1))
   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_train_loss+=loss.item()

   # Perform a backward pass to calculate the gradients.
   #compute gradient of loss w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter.
    loss.backward()

   # Clip the norm of the gradients to 1.0.
   # This is to help prevent the "exploding gradients" problem
   #within a specific range (clip)
    torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)

     
   # Update parameters and take a step using the computed gradient.
   # The optimizer dictates the "update rule"--how the parameters are
   # modified based on their gradients, the learning rate, etc.
   #updates all the parameters based on parameter.grad
    optimizer.step()

   # Update the learning rate.
    scheduler.step()
#Return to the epoch
   # Calculate the average loss over all of the batches.  
  avg_train_loss= total_train_loss/len(train_dataloader)
  print("Average train loss: {}".format(avg_train_loss))
  #append the loss for trainig
  loss_values.append(avg_train_loss)
  print(" Training is completed")
  print("Running Validation... ")

  #put the model in evaluation mode
  model1.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  predictions , true_labels = [], []

  for batch in valid_dataloader:

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
   
    #print("validation labels:",b_labels)
     #always clear the gradients
     #in evaluation mode, you can't do backprog, to save time, dont compute the gradients
    with torch.no_grad():
      #perform a forward pass
      loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    logits=logits.detach().cpu().numpy()
    label_ids=b_labels.to('cpu').numpy()
    label_ids=b_labels


   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_eval_loss+=loss.mean().item()
    predictions.extend([list(p) for p in np.argmax(logits,axis=2)])
    true_labels.extend(label_ids)

  eval_loss=total_eval_loss/len(valid_dataloader) 
  validation_loss_values.append(eval_loss)
  print("Validation loss: {}".format(eval_loss))

  pred_tags=[tag_values[p_i] for p,l in zip(predictions,true_labels) for p_i,l_i in zip(p,l) if tag_values[l_i] != "PAD"]

  valid_tags=[tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i]!="PAD"]

  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='weighted')))
  print("Classification Report : {}".format(classification_report(pred_tags,valid_tags)))

EPOCH:   0%|          | 0/2 [00:00<?, ?it/s]


Training...
0


RuntimeError: ignored

Demostrate the testing of sentence 

In [76]:
test_sentence = "Cerebral125 albumin was increased to similar proportions in those groups submitted to hyperosmolality."

In [77]:
#tokenize  the text
tokenized_sentence=scibert_tokenizer.encode(test_sentence)
#the output is sequence of ids (token to ids)

In [97]:
print(tokenized_sentence)

[101, 21831, 11964, 1571, 1312, 1394, 1108, 2569, 1106, 1861, 21136, 1107, 1343, 2114, 7402, 1106, 177, 24312, 2155, 3702, 14258, 1785, 119, 102]


In [98]:
#conver to tensor format
input_ids = torch.tensor([tokenized_sentence])

In [99]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [100]:
#this 
input_ids=input_ids.cuda()


In [ ]:
 #batch=tuple(t.to(device) for t in batch)

In [102]:
#run the sentence through the model
with torch.no_grad():
  logits=model1(input_ids)

In [ ]:
#run the sentence through the model
#with torch.no_grad():
#  logits_2=model2(input_ids)

In [ ]:
#label_indices_model2=torch.argmax(logits_2[0],axis=2)


In [103]:
label_indices=torch.argmax(logits[0],axis=2)


In [104]:
print(label_indices)

tensor([[0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0')


In [ ]:
#print(label_indices_model2)

In [105]:
#join the split tokens
tokens=scibert_tokenizer.convert_ids_to_tokens(input_ids[0])

In [106]:
print(tokens)

['[CLS]', 'cerebral', '##12', '##5', 'album', '##in', 'was', 'increased', 'to', 'similar', 'proportions', 'in', 'those', 'groups', 'submitted', 'to', 'h', '##yper', '##os', '##mo', '##lal', '##ity', '.', '[SEP]']


In [107]:
new_token,new_labels=[],[]
for token,label_idx in zip(tokens,label_indices[0]):
  if token.startswith("##"):
    #add at the end
    new_token[-1]=new_token[-1]+token[2:]
  else:
    #if its not special token
    new_labels.append(tag_values[label_idx])
    new_token.append(token) 

In [108]:
for token,label in zip(new_token,new_labels):
  print("{}\t{}".format(token,label))
  

[CLS]	O
cerebral125	B-DISEASE
albumin	O
was	O
increased	O
to	O
similar	O
proportions	O
in	O
those	O
groups	O
submitted	I-DRUGS
to	O
hyperosmolality	O
.	O
[SEP]	O
